## Capítulo 7. Manipulación de archivos Excel con pandas
pandas no se soporta en Excel para leer y escribir en archivos Excel. Es decir se puede tener acceso y modificar los archivos Excel sin tener el programa de Excel instalado.

### Caso de estudio: Reporte de Excel
En la carpeta "sales_data" se encuentran archivos Excel con transacciones de ventas simuladas para un proveedor de telecomunicaciones que vende diferentes planes (Bronce, Plata y Oro) en algunas tiendas de los Estados Unidos. Para cada mes hay dos archivos, uno en la subcarpeta "new" para nuevos contratos y otro en la subcarpeta "existing" para clientes existentes. Los reportes vienen en formatos distintos. Los clientes nuevos vienen en formato **xlsx** y los clientes existentes en formato **xls**. Cada uno de los archivos tiene hasta 10.000 transacciones y nuestra meta es producir un reporte Excel que muestre las ventas totales por tienda y por mes.

In [2]:
import pandas as pd

In [2]:
df = pd.read_excel("sales_data/new/January.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9493 entries, 0 to 9492
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    9493 non-null   object        
 1   store             9493 non-null   object        
 2   status            9493 non-null   object        
 3   transaction_date  9493 non-null   datetime64[ns]
 4   plan              9493 non-null   object        
 5   contract_type     9493 non-null   object        
 6   amount            9493 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 519.3+ KB


### Leyendo y escribiendo archivos Excel con pandas
El caso de estudio usó "read_excel" y "df.to_excel" con sus argumentos por defecto para mantener las cosas simples. Se mostrarán los argumentos y opciones más comúnmente usados cuando se leen y se escriben archivos Excel con pandas.

#### La función "read_excel" y la Clase ExcelFile


In [3]:
df = pd.read_excel("xl/stores.xlsx", 
    sheet_name="2019", skiprows=1, usecols="B:F")
df 

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING
2,Chicago,4,Katelin,2020-01-31,NaN
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   5 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 368.0+ bytes


In [5]:
def fix_missing(x):
    return False if x in ["", "MISSING"] else x

In [6]:
df = pd.read_excel("xl/stores.xlsx", 
                   sheet_name="2019", skiprows=1, usecols="B:F", 
                   converters={"Flagship": fix_missing})
df

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,False
2,Chicago,4,Katelin,2020-01-31,False
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   6 non-null      bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 326.0+ bytes


La función "read_excel" acepta una lista de nombres de hojas de cálculo. En este caso, la función retorna un diccionario con los DataFrame como valores y el nombre de la hoja como llave. Para leer en todas las hojas, se debería proveer **sheet_name=none**. Note la ligera variación de cómo se usa **usecols** proveyendo los nombres de las columnas de la tabla:

In [8]:
sheets = pd.read_excel("xl/stores.xlsx", sheet_name=["2019", "2020"], 
                       skiprows=1, usecols=["Store", "Employees"])
sheets["2019"].head()

,Store,Employees
0,New York,10
1,San Francisco,12
2,Chicago,4
3,Boston,5
4,Washington DC,3


In [9]:
df = pd.read_excel("xl/stores.xlsx", sheet_name=0, 
                   skiprows=2, skipfooter=3, 
                   usecols="B:C,F", header=None, 
                   names=["Branch", "Employee_Count", "Is_Flagship"])
df

,Branch,Employee_Count,Is_Flagship
0,New York,10,False
1,San Francisco,12,MISSING
2,Chicago,4,NaN


In [10]:
df = pd.read_excel("xl/stores.xlsx", sheet_name="2019", 
                   skiprows=1, usecols="B,C,F", skipfooter=2, 
                   na_values="MISSING", keep_default_na=False)
df

,Store,Employees,Flagship
0,New York,10,False
1,San Francisco,12,NaN
2,Chicago,4,
3,Boston,5,True


En Python la cláusula **with** se usa para manejar recursos como archivos y conexiones a bases de datos.

In [11]:
with open("output.txt", "w") as f:
    f.write("Some text") 
#Cuando se sale de **with** el "manejador de contexto" cierra el archivo.

In [12]:
#La clase ExcelFile en acción.
with pd.ExcelFile("xl/stores.xls") as f:
    df1 = pd.read_excel(f, "2019", skiprows=1, usecols="B:F", nrows=2)
    df2 = pd.read_excel(f, "2020", skiprows=1, usecols="B:F", nrows=2)
df1

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


In [13]:
#La clase ExcelFile da acceso a la lista con los nombres de todas las hojas del libro.
stores = pd.ExcelFile("xl/stores.xlsx")
stores.sheet_names

['2019', '2020', '2019-2020']

In [14]:
#En vez de una ruta de archivo, se puede proveer una dirección url para accesar un archivo Excel.
url = ("https://raw.githubusercontent.com/fzumstein/"
       "python-for-excel/1st-edition/xl/stores.xlsx")
pd.read_excel(url, skiprows=1, usecols="B:E", nrows=2)

,Store,Employees,Manager,Since
0,New York,10,Sarah,2018-07-20
1,San Francisco,12,Neriah,2019-11-02


In [17]:
#Los archivos Excel con formatos "xlsb" son archivos Excel binarios.
#Requieren de mucho menos espacio para el almacenamiento.
pd.read_excel("xl/stores.xlsb", engine="pyxlsb", skiprows=1, usecols="B,C,F")


,Store,Employees,Flagship
0,New York,10,False
1,San Francisco,12,MISSING
2,Chicago,4,NaN
3,Boston,5,True
4,Washington DC,3,False
5,Las Vegas,11,False


#### El método .to_excel y la clase ExcelWriter
La forma más fácil de escribir un archivo Excel con pandas es usar su método df.to_excel, el cual le permite especificar la celda de la hoja de cálculo a la cual se desea escribir el DataFrame.
También se decide si se incluye o no, los encabezados de las columnasy el índice del DataFrame; y además, cómo tratar con tipos de datos como np.nan y np.inf, que no tienen representación equivalente en Excel.


In [18]:
import numpy as np
import datetime as dt


In [19]:
data = [[dt.datetime(2020,1,1, 10, 13), 2.222, 1, True], 
        [dt.datetime(2020,1,2), np.nan, 2, False], 
        [dt.datetime(2020,1,2), np.inf, 3, True]]
df = pd.DataFrame(data=data, columns=["Dates", "Floats", "Integers", "Booleans"])
df.index.name="index"
df

,Dates,Floats,Integers,Booleans
index,,,,
0,2020-01-01 10:13:00,2.222,1,True
1,2020-01-02 00:00:00,NaN,2,False
2,2020-01-02 00:00:00,inf,3,True


In [20]:
df.to_excel("written_with_pandas.xlsx", sheet_name="Output", 
            startrow=1, startcol=1, index=True, header=True, 
            na_rep="<NA>", inf_rep="<INF>")

In [21]:
#Si se desea escribir múltiples DataFrames en una misma hoja de cálculo o en 
# diferentes hojas de cálculo, se debe usar la clase ExcelWriter.
with pd.ExcelWriter("written_with_pandas2.xlsx") as writer:
    df.to_excel(writer, sheet_name="Sheet1", startrow=1, startcol=1)
    df.to_excel(writer, sheet_name="Sheet1", startrow=10, startcol=1)
    df.to_excel(writer, sheet_name="Sheet2") 

#### Limitaciones cuando se usa pandas con archivos Excel
* No se puede incluir un título o un gráfico.
* No hay forma de cambiar el formato por defecto del encabezado y del índice en Excel.
* Cuando lee archivos, pandas automáticamente transforma las celdas con errores como #REF o #NUM en NaN, haciendo imposible buscar por errores específicos en la hoja de cálculo.
* Trabajar con archivos Excel muy grandes requiere de ajustes extras.